# Building A RAG System with Gemma, Elasticsearch and Open Source Models

Authored By: [lloydmeta](https://huggingface.co/lloydmeta)

In [1]:
from huggingface_hub import notebook_login

notebook_login()

## Step 1: Installing Libraries


The shell command sequence below installs libraries for leveraging open-source large language models (LLMs), embedding models, and database interaction functionalities. These libraries simplify the development of a RAG system, reducing the complexity to a small amount of code:


- Elasticsearch: A Python library for interacting with Elasticsearch, along with nice pythonic wrappers.
- Hugging Face datasets: Holds audio, vision, and text datasets
- Hugging Face Accelerate: Abstracts the complexity of writing code that leverages hardware accelerators such as GPUs. Accelerate is leveraged in the implementation to utilise the Gemma model on GPU resources.
- Hugging Face Transformers: Access to a vast collection of pre-trained models
- Hugging Face Sentence Transformers: Provides access to sentence, text, and image embeddings.

In [ ]:
!pip install datasets elasticsearch sentence_transformers transformers
# Install below if using GPU
!pip install accelerate

## Step 2: Data sourcing and preparation


The data utilised in this tutorial is sourced from Hugging Face datasets, specifically the
[MongoDB/embedded_movies dataset](https://huggingface.co/datasets/MongoDB/embedded_movies).

In [13]:
# Load Dataset
from datasets import load_dataset

# https://huggingface.co/datasets/MongoDB/embedded_movies
dataset = load_dataset("MongoDB/embedded_movies")

dataset

DatasetDict({
    train: Dataset({
        features: ['plot_embedding', 'writers', 'cast', 'genres', 'poster', 'languages', 'directors', 'num_mflix_comments', 'imdb', 'countries', 'plot', 'fullplot', 'type', 'rated', 'awards', 'title', 'runtime', 'metacritic'],
        num_rows: 1500
    })
})

The operations within the following code snippet below focus on enforcing data integrity and quality.
1. The first process ensures that each data point's `fullplot` attribute is not empty, as this is the primary data we utilise in the embedding process.
2. This step also ensures we remove the `plot_embedding` attribute from all data points as this will be replaced by new embeddings created with a different embedding model, the `gte-large`.

In [15]:
# Data Preparation

# Remove data point where plot coloumn is missing
dataset = dataset.filter(lambda x: x["fullplot"] is not None)

# Remove the plot_embedding from each data point in the dataset as we are going to create new embeddings with an open source embedding model from Hugging Face
# dataset = dataset.remove_columns("plot_embedding")
dataset["train"]

Filter:   0%|          | 0/1452 [00:00<?, ? examples/s]

Dataset({
    features: ['writers', 'cast', 'genres', 'poster', 'languages', 'directors', 'num_mflix_comments', 'imdb', 'countries', 'plot', 'fullplot', 'type', 'rated', 'awards', 'title', 'runtime', 'metacritic'],
    num_rows: 1452
})

## Step 3: Generating embeddings

**The steps in the code snippets are as follows:**
1. Import the `SentenceTransformer` class to access the embedding models.
2. Load the embedding model using the `SentenceTransformer` constructor to instantiate the `gte-large` embedding model.
3. Define the `get_embedding` function, which takes a text string as input and returns a list of floats representing the embedding. The function first checks if the input text is not empty (after stripping whitespace). If the text is empty, it returns an empty list. Otherwise, it generates an embedding using the loaded model.
4. Generate embeddings by applying the `get_embedding` function to the "fullplot" column of the `dataset_df` DataFrame, generating embeddings for each movie's plot. The resulting list of embeddings is assigned to a new column named embedding.

*Note: It's not necessary to chunk the text in the full plot, as we can ensure that the text length remains within a manageable range.*



In [3]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")

def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def add_fullplot_embedding(x):
    full_plots = x["fullplot"]
    return { "embedding": [ get_embedding(full_plot) for full_plot in full_plots] }


dataset = dataset.map(add_fullplot_embedding, batched = True)
dataset["train"]

## Step 4: Database setup and connection

Elasticsearch acts as both an operational and a vector database. It offers a database solution that efficiently stores, queries and retrieves vector embeddings—the advantages of this lie in the simplicity of database maintenance, management and cost.

**To create a new Elasticsearch database, set up a database cluster:**

1. Head over to Elastic's official site and register for a [free Cloud account](http://cloud.elastic.co), or for existing users, [sign into Elastic Cloud](http://cloud.elastic.co).

2. Create a new ES cluster

3. After successfully creating the cluster, note down the `elastic` user password and Cloud ID, and copy these into the Colab secrets environment in variables called `ELASTIC_PASSWORD` AND `CLOUD_ID` respectively.


## Step 6: Establish Data Connection

The code snippet below also utilises the elasticsearch lib to create an `Elasticsearch` client object, representing the connection to the cluster.

In [39]:
from elasticsearch import Elasticsearch, helpers
from google.colab import userdata

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD =  userdata.get("ELASTIC_PASSWORD") # or "<YOUR PASSWORD>"

# Found in the 'Manage Deployment' page
CLOUD_ID = userdata.get("CLOUD_ID") # or "<YOUR ELASTIC CLOUD CLOUD_ID>"

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

# Successful response!
client.info()

ObjectApiResponse({'name': 'instance-0000000000', 'cluster_name': '381fbb2e86b047a18c0aff54ae9bab9a', 'cluster_uuid': 'McJf_GKTRge-_QOQqHY1Tg', 'version': {'number': '8.12.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '48a287ab9497e852de30327444b0809e55d46466', 'build_date': '2024-02-19T10:04:32.774273190Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})



## Step 5: Create a Search Index with vector search mappings.

At this point, we create an index in Elasticsearch with the right index mappings to handle vector searches.

Go here to read more about [Elasticsearch vector capabilities](https://www.elastic.co/what-is/vector-search).


The `1024` value of the `dims` field corresponds to the dimension of the vector generated by the gte-large embedding model. If you use the `gte-base` or `gte-small` embedding models, the numDimension value in the vector search index must be set to 768 and 384, respectively.


In [26]:
index_mapping = {
    "properties": {
      "embedding": {
          "type": "dense_vector",
          "dims": 1024,
          "index": "true",
          "similarity": "cosine"
      },
      "fullplot": { "type": "text" },
      "plot": { "type": "text" },
      "title": { "type": "text" }
    }
}

client.indices.create(index="movies", mappings=index_mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies'})

Ingesting data into a Elasticsearch is best done in batches. Luckily `helpers` offers an esasy way to do this.

In [40]:
def batch_to_bulk_actions(batch):
    for record in batch:
        yield {
            "_index": "movies",
            "_source": {
                "title" : record["title"],
                "fullplot" : record["fullplot"],
                "plot" : record["plot"],
                "embedding" : record["embedding"]
            }
        }

def bulk_index(ds):
  start = 0
  end = len(ds)
  batch_size = 100
  for batch_start in range(start, end, batch_size):
      batch_end = min(batch_start + batch_size, end)
      batch = ds.select(range(batch_start, batch_end))
      actions = batch_to_bulk_actions(batch)
      helpers.bulk(client, actions)

bulk_index(dataset["train"])

print("Data ingestion into Elasticsearch complete!")

DatasetDict({
    train: Dataset({
        features: ['writers', 'cast', 'genres', 'poster', 'languages', 'directors', 'num_mflix_comments', 'imdb', 'countries', 'plot', 'fullplot', 'type', 'rated', 'awards', 'title', 'runtime', 'metacritic', 'embedding'],
        num_rows: 1452
    })
})
end [1452]


## Step 7: Perform Vector Search on User Queries

The following step implements a function that returns a vector search result by generating a query that contains an embedded form of your text query.

In [24]:
def vector_search(plot_query):
    question_embedding = get_embedding(plot_query)
    response = client.search(
       index = "movies",
       knn={
           "field": "embedding",
           "query_vector":  question_embedding,
           "k": 10,
           "num_candidates": 150
       },
       size = 5
    )
    results = []
    for hit in response['hits']['hits']:
        id = hit['_id']
        score = hit['_score']
        title = hit['_source']['title']
        plot = hit['_source']['plot']
        fullplot = hit['_source']['fullplot']
        result = { "id": id, "_score": score, "title": title, "plot": plot, "fullplot": fullplot }
        results.append(result)
    return results

def pretty_search(query):

    get_knowledge = vector_search(query)

    search_result = ""
    for result in get_knowledge:
        search_result += f"Title: {result.get('title', 'N/A')}, Plot: {result.get('fullplot', 'N/A')}\n"

    return search_result

## Step 8: Handling user queries and loading Gemma


In [33]:
# Conduct query with retrival of sources
def combined_query(query):
  source_information = pretty_search(query)
  return f"Query: {query}\nContinue to answer the query by using these Search Results:\n{source_information}."

query = "What is the best romantic movie to watch and why?"
combined_results = combined_query(query)

print(combined_results)

Query: What is the best romantic movie to watch and why?
Continue to answer the query by using these Search Results:
Title: Shut Up and Kiss Me!, Plot: Ryan and Pete are 27-year old best friends in Miami, born on the same day and each searching for the perfect woman. Ryan is a rookie stockbroker living with his psychic Mom. Pete is a slick surfer dude yet to find commitment. Each meets the women of their dreams on the same day. Ryan knocks heads in an elevator with the gorgeous Jessica, passing out before getting her number. Pete falls for the insatiable Tiara, but Tiara's uncle is mob boss Vincent Bublione, charged with her protection. This high-energy romantic comedy asks to what extent will you go for true love?
Title: Pearl Harbor, Plot: Pearl Harbor is a classic tale of romance set during a war that complicates everything. It all starts when childhood friends Rafe and Danny become Army Air Corps pilots and meet Evelyn, a Navy nurse. Rafe falls head over heels and next thing you kn

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# CPU Enabled uncomment below 👇🏽
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
# GPU Enabled use below 👇🏽
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [43]:
def rag_query(query):
  combined_information = combined_query(query)

  # Moving tensors to GPU
  input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
  response = model.generate(**input_ids, max_new_tokens=700)

  return tokenizer.decode(response[0], skip_special_tokens = True)


print(rag_query("What's a romantic movie that I can watch with my wife?"))

Query: What's a romantic movie that I can watch with my wife?
Continue to answer the query by using these Search Results:
Title: King Solomon's Mines, Plot: Guide Allan Quatermain helps a young lady (Beth) find her lost husband somewhere in Africa. It's a spectacular adventure story with romance, because while they fight with wild animals and cannibals, they fall in love. Will they find the lost husband and finish the nice connection?
Title: Shut Up and Kiss Me!, Plot: Ryan and Pete are 27-year old best friends in Miami, born on the same day and each searching for the perfect woman. Ryan is a rookie stockbroker living with his psychic Mom. Pete is a slick surfer dude yet to find commitment. Each meets the women of their dreams on the same day. Ryan knocks heads in an elevator with the gorgeous Jessica, passing out before getting her number. Pete falls for the insatiable Tiara, but Tiara's uncle is mob boss Vincent Bublione, charged with her protection. This high-energy romantic comedy 